In [40]:
import pandas as pd

In [41]:
data=pd.read_csv(r"C:\Users\Varun\Music\Python\playground-series-s3e8\train.csv")

In [ ]:
data=pd.read_csv(r"C:\Users\Varun\Music\Python\playground-series-s3e8\train.csv")

In [ ]:
data.info()
data.drop(labels=["id"],axis=1,inplace=True)
data

In [ ]:
data.duplicated().sum()

In [44]:
cat_columns=data.columns[data.dtypes=="object"]
num_columns=data.columns[data.dtypes!="object"]

In [ ]:
data[num_columns].describe().T

In [ ]:
for col in cat_columns:
    print(data[col].value_counts())
    print("=================")

In [47]:
# Ordianal Encoding
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map={"I1":1,"SI2":2,"SI3":3,"VS2":4,"VS1":5,"VVS2":6,"VVS1":7,"IF":8}
color_map={"D":1,"E":2,"F":3,"G":4,"H":5,"I":6,"J":7}

In [48]:
data["cut"]=data["cut"].map(cut_map)
data["clarity"]=data["clarity"].map(clarity_map)
data["color"]=data["color"].map(color_map)

In [49]:
from sklearn.impute import SimpleImputer # Handling Missing Values
from sklearn.preprocessing import StandardScaler # Handling Feature Selection
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [50]:
X=data.drop(labels=["price"],axis=1)
Y=data["price"]

In [51]:
cat_cols=X.select_dtypes(include="object").columns
num_cols=X.select_dtypes(exclude="object").columns

In [52]:
cut_categories={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_categories={"I1":1,"SI2":2,"SI3":3,"VS2":4,"VS1":5,"VVS2":6,"VVS1":7,"IF":8}
color_categories={"D":1,"E":2,"F":3,"G":4,"H":5,"I":6,"J":7}

In [53]:
num_pipeline=Pipeline(
    steps=[
        ("Imputer",SimpleImputer()),
        ("Scalar",StandardScaler())
    ]
)

In [54]:
cat_pipeline=Pipeline(
    steps=[
        ("Imputer",SimpleImputer(strategy="most_frequent")),
        ("Ordinal Encoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)


In [55]:
preprocessor=ColumnTransformer(
    [
        ("Numerical Pipeline",num_pipeline,num_cols),
        ("Categorical Pipeline",cat_pipeline,cat_cols)
    ]
)

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
preprocessor.fit_transform(X_train)

In [58]:
# Model Training

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [59]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  # You can change 'mean' to 'median' or 'most_frequent'
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)


In [60]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'RandomForest':RandomForestRegressor(),
    'XGBoost':XGBRegressor()
}

In [61]:
train_model_list=[]
model_list=[]
r2_list=[]

In [ ]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

In [ ]:
def evaluate_model(true,pred):
    r2=r2_score(true,pred)
    mae=mean_absolute_error(true,pred)
    mse=mean_squared_error(true,pred)
    return mae,mse,r2


for i in range(len(list(models))):
    model=list(models.values())[i]
    # Train the model
    model.fit(X_train, Y_train)
    # Make prediction
    Y_pred=model.predict(X_test)
    # For validation
    mse,mae,r2=evaluate_model(Y_test,Y_pred)
    # Model training performance
    print("Model Training Performance")
    print("MSE:",mse)
    print("MAE:",mae)
    print("R2 Score:",r2)
    r2_list.append(r2)
    print("="*40)
    print("\n")